# Scraping sitio "Books To Scrape"
### https://books.toscrape.com/

### - Imports

In [8]:
from bs4 import BeautifulSoup
from sqlalchemy import Column, Integer, String, Boolean, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

import requests
import os

### - Variables globales

In [ ]:
link = 'https://books.toscrape.com/'

### - Clases auxiliares

In [ ]:
class Guardar():
    carpeta_contenedora_de_archivos = 'datos'

    def categorias_como_csv(self, filename, datos):
        if self._carpeta_existe(self.carpeta_contenedora_de_archivos):
            ruta = os.path.join(self.carpeta_contenedora_de_archivos, filename)
        else:
            os.makedirs(self.carpeta_contenedora_de_archivos)
            ruta = os.path.join(self.carpeta_contenedora_de_archivos, filename)
        if self._archivo_existe(ruta):
            print(f"Open: {ruta}.csv...")
        else:
            print(f"Creating: {ruta}.csv...")
            print(f"Succesfully: {ruta}.csv creado con éxito.")
            print(f"Open: {ruta}.csv...")

        with open(self.carpeta_contenedora_de_archivos + "/" + filename + '.csv', 'w') as archivo:
            print("Writing: categoria,link")
            archivo.write("categoria,link\n")
            for key, value in datos.items():
                print(f"Writing: {key},{value}")
                archivo.write(f"{key},{value}{'\n' if key != list(datos.keys())[-1] else ''}")
            print(f"Message: Datos cargados al archivo {ruta}.csv con éxito.")
            print(f"Closed: Archivo {ruta}.csv")

    def libros_como_csv(self, filename, datos):
        if self._carpeta_existe(self.carpeta_contenedora_de_archivos):
            ruta = os.path.join(self.carpeta_contenedora_de_archivos, filename)
        else:
            os.makedirs(self.carpeta_contenedora_de_archivos)
            ruta = os.path.join(self.carpeta_contenedora_de_archivos, filename)
        if self._archivo_existe(ruta):
            print(f"Open: {ruta}.csv...")
        else:
            print(f"Creating: {ruta}.csv...")
            print(f"Succesfully: {ruta}.csv creado con éxito.")
            print(f"Open: {ruta}.csv...")

        with open(ruta + '.csv', 'w') as archivo:
            print("Writing: categoria,link")
            archivo.write("categoria,link\n")
            for key, value in datos.items():
                # print("Guardando: ", key, value)
                for item in value:
                    print(f"Writing: {key},{value}")
                    archivo.write(f"{key},{item['link']}{'\n' if key != list(datos.keys())[-1] else ''}")
                    
            print(f"Message: Datos cargados al archivo {ruta}.csv con éxito.")
            print(f"Closed: Archivo {ruta}.csv")

    def informacion_libros_como_csv(self, filename, datos):
        if self._carpeta_existe(self.carpeta_contenedora_de_archivos):
            ruta = os.path.join(self.carpeta_contenedora_de_archivos, filename)
        else:
            os.makedirs(self.carpeta_contenedora_de_archivos)
            ruta = os.path.join(self.carpeta_contenedora_de_archivos, filename)
        if self._archivo_existe(ruta):
            print(f"Open: {ruta}.csv...")
        else:
            print(f"Creating: {ruta}.csv...")
            print(f"Succesfully: {ruta}.csv creado con éxito.")
            print(f"Open: {ruta}.csv...")

        with open(ruta + '.csv', 'w', encoding='utf-8', errors='replace') as archivo:
            print("Writing: categoria,titulo,precio,en_stock,rating,imagen,descripcion")
            archivo.write("categoria,titulo,precio,en_stock,rating,imagen,descripcion\n")
            for i, data in enumerate(datos):
                d = f"{data['categoria']},{data['titulo']},{data['precio']},{data['en_stock']},{data['rating']},{data['imagen']},{data['descripcion']}"
                print(f"Writing: {d}")
                archivo.write(f"{d}{'\n' if i < len(datos-1) else ''}")
                                                                       
                # for key, value in data.items():
                #     print(f"Writing: {key},{value}")
                #     archivo.write(f"{key},{item['link']}{'\n' if key != list(datos.keys())[-1] else ''}")
                    
            print(f"Message: Datos cargados al archivo {ruta}.csv con éxito.")
            print(f"Closed: Archivo {ruta}.csv")


    def _archivo_existe(self, filename):
        return os.path.exists(filename + ".csv")
    
    def _carpeta_existe(self, carpeta):
        return os.path.exists(carpeta)



class Descarga:
    def descargar(self, url:str) -> str:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.text
        except requests.exceptions.RequestException as e:
            raise(f"ErrorDescarga: {e}")



In [ ]:
class Scraping():
    def __init__(self, link:str):
        self.link = link
        self.html = BeautifulSoup(Descarga().descargar(self.link), 'html.parser')
        self._categorias = {}
        self._libros = {}
        self._informacion_libros = []
        
    def buscar_categorias(self) -> None:
        """Busca las categorias de la pagina web"""
        print(f"Connect: {self.link}")
        print("Getting data...")
        categorias = self.html.find('ul', class_='nav nav-list').find_all('li')
        for categoria in categorias:
            categoria_key = f'{categoria.find('a').text.strip()}'
            categoria_link = f'{self.link}/{categoria.find("a").get("href")}'
            print(f"Extracting: Categoria({categoria_key}), link({categoria_link})")
            self._categorias[categoria_key] = categoria_link
        del self._categorias['Books']

    def buscar_libros(self):
        """Busca los libros de la pagina web"""
        for _, (categoria, link) in enumerate(self._categorias.items()):
            full_link = link

            print(f"Connect: {full_link}")
            html = BeautifulSoup(Descarga().descargar(full_link), 'html.parser')
            
            print(f"Getting data...")
            # Busca los libros en el index de cada categoria
            libros = html.find('ol', class_='row').find_all('li')

            tmp_libros = []
            for libro in libros:
                data = f'{self.link}/catalogue{libro.find("a")["href"].split('/..')[-1]}'
                print(f"Extracting: {data}")
                tmp_libros.append({
                    'link': data,
                    })
            self._libros[categoria] = tmp_libros

            tiene_indice = html.find('div', class_='col-sm-8 col-md-9').find('li', class_='current')
            # tiene_indice = html.find('li', class_='current')
            if tiene_indice:
                print(f"Have index...")
                indice = tiene_indice.text.split()[-1]
                print(f"Getting index from {categoria}...")
                for i in range(1, int(indice) + 1):
                    full_link = f'{full_link}?page={i}.html'
                    print(f"Connect: {full_link}")
                    html = BeautifulSoup(Descarga().descargar(full_link), 'html.parser')
        
                    # Busca los libros en el index de cada categoria
                    libros = html.find('ol', class_='row').find_all('li')
                    print("Getting data...")
                    tmp_libros = []
                    for libro in libros:
                        data = f'{self.link}/catalogue{libro.find("a")["href"].split('/..')[-1]}'
                        print(f"Extracting: {data}")
                        tmp_libros.append({
                            'link': data,
                            })
                    self._libros[categoria] = tmp_libros

    def buscar_libro_individual(self):
        """Busca los libros de la pagina web"""
        pass

    def buscar_libros_del_link(self):
        """Busca los libros de la pagina web"""
        for categoria, links in self._libros.items():
            # print(categoria, link)
            for link in links:
                link = link['link']
                print(f"Connect: {categoria} {link}")
                html = BeautifulSoup(Descarga().descargar(link), 'html.parser')

                print("Extracting information...")
                title = html.find('article', class_='product_page').find('h1').text.strip()
                precio = html.find('p', class_='price_color').text.strip()
                en_stock = html.find('p', class_='instock').find('i', class_='icon-ok').text.strip()
                rating = html.find('p', class_='star-rating').attrs['class'][1]
                imagen_link = self.link + '/' + html.find('div', class_='item').find('img').attrs['src'].split('../')[-1]
                existe_etiqueta_descripcion = html.find('div', id='product_description')
                if existe_etiqueta_descripcion and existe_etiqueta_descripcion.next_sibling:
                    descripcion = html.find('div', id='product_description').find_next_sibling('p').text
                else:
                    descripcion = None

                print(f"Data: {categoria}, {title}, {precio}, {en_stock}, {rating}, {imagen_link}, {descripcion}")

                self._informacion_libros.append({
                    'categoria': categoria,
                    'titulo': title,
                    'precio': precio,
                    'en_stock': en_stock,
                    'rating': rating,
                    'imagen': imagen_link,
                    'descripcion': descripcion,
                })
                print(self._informacion_libros)

    def run(self):
        """Ejecuta el programa"""
        self.buscar_categorias()
        # Guardar().categorias_como_csv('categorias', self._categorias)
        self.buscar_libros()
        # Guardar().libros_como_csv('libros', self._libros)
        self.buscar_libros_del_link()
        # Guardar().informacion_libros_como_csv('informacion_libros', self._informacion_libros)



### - Descargar Autores utilizando 

In [ ]:

def buscar_autor_por_titulo(titulo):
    # Endpoint de búsqueda por título
    url = 'https://openlibrary.org/search.json'
    params = {'title': titulo}
    
    respuesta = requests.get(url, params=params)
    datos = respuesta.json()
    
    # La lista de documentos devueltos
    docs = datos.get('docs', [])
    if not docs:
        return None
    
    # Extraemos el primer autor del primer resultado
    autores = docs[0].get('author_name', [])
    return autores[0] if autores else None

# # Ejemplo de uso
# titulos = ["A Light in the Attic", "You can't bury them all: Poems"]

# for titulo in titulos:
#     autor = buscar_autor_por_titulo(titulo)
#     print(f"Autor de «{titulo}»: {autor}")



### - Modelos de tablas

In [4]:
Base = declarative_base()

In [11]:
# class Categorias(Base):
#     __tablename__ = 'categorias'
#     id = Column(Integer, primary_key=True)
#     nombre = Column(String)
#     link = Column(String)

# class Libros(Base):
#     __tablename__ = 'libros'
#     id =  Column(Integer, primary_key=True)
#     titulo = Column(String)
#     precio = Column(String)
#     descripcion = Column(String)
#     imagen = Column(String)
#     id_categoria = Column(Integer)

# class Stock(Base):
#     __tablename__ = 'stock'
#     id = Column(Integer, primary_key=True)
#     en_stock = Column(Boolean)
#     cantidad = Column(String)
#     id_libro = Column(Integer)

from sqlalchemy import Column, Integer, String, Boolean, ForeignKey
from sqlalchemy.orm import relationship, declarative_base

Base = declarative_base()

class Categoria(Base):
    __tablename__ = 'categorias'
    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    link = Column(String)

    # Relación inversa opcional
    libros = relationship('Libro', back_populates='categoria')

class Libro(Base):
    __tablename__ = 'libros'
    id = Column(Integer, primary_key=True)
    titulo = Column(String)
    precio = Column(String)
    descripcion = Column(String)
    imagen = Column(String)

    id_categoria = Column(Integer, ForeignKey('categorias.id'))
    categoria = relationship('Categoria', back_populates='libros')

    stock = relationship('Stock', back_populates='libro', uselist=False)

class Stock(Base):
    __tablename__ = 'stock'
    id = Column(Integer, primary_key=True)
    en_stock = Column(Boolean)
    cantidad = Column(String)

    id_libro = Column(Integer, ForeignKey('libros.id'))
    libro = relationship('Libro', back_populates='stock')


### - Base de Datos

In [13]:
engine = create_engine('sqlite:///libros.db')

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()



### - Main - Implementación

In [ ]:
scraping = Scraping(link)
scraping.run()   

### - Manejar recursos

In [3]:
class HTML():
    def get_text(self, link:str) -> str:
        try:
            return requests.get(link).text
        except requests.exceptions.MissingSchema as e:
            print(e)

class Parser():
    def parser(self, html):
        return BeautifulSoup(html, 'html.parser')

class ScrapingBooksToScrape():
    def __init__(self, link:str, recurso:HTML, parser:Parser) -> None:
        self._link_principal = link
        self._recurso_html = recurso
        self._parser = parser
        self._lista_links_principal = []
        self._lista_links_libros = []


    def busqueda_links_principal(self, classname:str) -> None:
        recurso = self._recurso_html.get_text(self._link_principal)

        if recurso:
            html = self._parser.parser(recurso)
            html_categorias = html(class_=classname)

            for categoria in html_categorias:
                enlaces = categoria.find_all('a')
                for link in enlaces:
                    self._lista_links_principal.append({
                        'categoria': link.text.strip(),
                        'link': link.get('href')
            })

    def busqueda_links_libros(self) -> None:
        print(len(self._lista_links_principal))
        for i, link in enumerate(self._lista_links_principal):
            full_url = self._link_principal + '/' + link['link']

            recurso = self._recurso_html.get_text(full_url)

            if recurso:
                html = self._parser.parser(recurso)

                tags_ol = html.find('ol', class_='row')

                for contenido in tags_ol:
                    print(type(contenido))
            
            break

                # etiqueta_hijo = html.find('div', class_='page-header')
                # etiqueta_padre = etiqueta_hijo.find_parent()

                # tags_ol_class_row = etiqueta_padre.find_all('ol', class_='row')
                # tags_li_hijo_tag_ol_class_row = tags_ol_class_row[0].find('li')

                # print(self._lista_links_principal[i]['categoria'], ': ', tags_li_hijo_tag_ol_class_row.find('a').get('href'))

                # for tag_li in tags_li_hijo_tag_ol_class_row:
                #     tag_li.find('a')
                


                # self._lista_links_libros.append(html)

    def get_lista_links_principal(self) -> list[dict]:
        print(self._link_principal)
        return self._lista_links_principal
    
    def get_lista_links_libros(self):
        return self._lista_links_libros
    

        

spider = ScrapingBooksToScrape(link_to_scrape, HTML(), Parser())
spider.busqueda_links_principal('side_categories')
spider.get_lista_links_principal()

# spider.busqueda_links_libros()
# spider.get_lista_links_libros()

https://books.toscrape.com/


[{'categoria': 'Books', 'link': 'catalogue/category/books_1/index.html'},
 {'categoria': 'Travel',
  'link': 'catalogue/category/books/travel_2/index.html'},
 {'categoria': 'Mystery',
  'link': 'catalogue/category/books/mystery_3/index.html'},
 {'categoria': 'Historical Fiction',
  'link': 'catalogue/category/books/historical-fiction_4/index.html'},
 {'categoria': 'Sequential Art',
  'link': 'catalogue/category/books/sequential-art_5/index.html'},
 {'categoria': 'Classics',
  'link': 'catalogue/category/books/classics_6/index.html'},
 {'categoria': 'Philosophy',
  'link': 'catalogue/category/books/philosophy_7/index.html'},
 {'categoria': 'Romance',
  'link': 'catalogue/category/books/romance_8/index.html'},
 {'categoria': 'Womens Fiction',
  'link': 'catalogue/category/books/womens-fiction_9/index.html'},
 {'categoria': 'Fiction',
  'link': 'catalogue/category/books/fiction_10/index.html'},
 {'categoria': 'Childrens',
  'link': 'catalogue/category/books/childrens_11/index.html'},
 {'c